## ProteinGym Processing Script
- The goal of this script is to extract metadata for all proteins found within ProteinGym
- This metadata is scraped from UniProt (covers 186/187 proteins). 

In [2]:
import pandas as pd
import requests
import time
import json

1. Load DF describing DMS assays

In [4]:
# https://raw.githubusercontent.com/OATML-Markslab/ProteinGym/main/reference_files/DMS_substitutions.csv
DMS_summary = pd.read_csv('../data/DMS_substitutions.csv')

2. We require "Accession" numbers to access the uniprot API - let's map our uniprot ID's to these

In [5]:
def get_uniprot_accessions(uniprot_ids):
    # URL for the UniProt ID mapping service
    url = "https://rest.uniprot.org/idmapping/run"
    
    # Prepare the data for the form submission
    form_data = {
        'from': 'UniProtKB_AC-ID',
        'to': 'UniProtKB',
        'ids': ",".join(uniprot_ids)  # joins the list of UniProt IDs into a single string separated by commas
    }
    
    # Send the request
    response = requests.post(url, data=form_data)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Extract the job ID from the response
        job_id = response.json().get('jobId')
    else:
        # Handle potential errors (simple print statement here, could be logging or raising an exception)
        print("Failed to submit job:", response.status_code, response.text)
        return None
    
    while True:
        url = f"https://rest.uniprot.org/idmapping/stream/{job_id}"

        # Send the GET request
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # You could return the response text directly or process it as needed
            raw_json = json.loads(response.text)
            mapping = {item['from']: item['to'] for item in raw_json['results']}

            return mapping

        time.sleep(1)

uniprot_ids = DMS_summary['UniProt_ID'].unique()
uniprot_mapping = get_uniprot_accessions(uniprot_ids)
DMS_summary['UniProt_Accession_id'] = DMS_summary['UniProt_ID'].apply(lambda x: uniprot_mapping.get(x, pd.NA))

In [6]:
DMS_summary = DMS_summary[DMS_summary['taxon'] == 'Human']

In [7]:
def get_protein_features(protein_id):

    requestURL = f"https://www.ebi.ac.uk/proteins/api/features/{protein_id}"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        return None

    features = json.loads(r.text)['features']

    features = [feature for feature in features if feature['category'] == 'PTM' or feature['type'] == 'BINDING']

    return features

DMS_summary['uniprot_features'] = DMS_summary['UniProt_Accession_id'].apply(lambda x: get_protein_features(x))

3. Now we can add explode our data with all possible substitutions from the DMS assays:

In [9]:
def process_DMS(row):
    # https://marks.hms.harvard.edu/proteingym/DMS_ProteinGym_substitutions.zip

    substitutions = pd.read_csv(f"../data/DMS_ProteinGym_substitutions/{row['DMS_filename']}")['mutant'].tolist()
    substitution_df = pd.DataFrame({
        'UniProt_Accession_id': row['UniProt_Accession_id'],
        'mutation': substitutions
    })

    substitution_df = substitution_df[~substitution_df['mutation'].str.contains(':')]

    substitution_df['POS'] = substitution_df['mutation'].apply(lambda x: int(x[1:-1]))
    substitution_df['REF'] = substitution_df['mutation'].apply(lambda x: x[0])
    substitution_df['ALT'] = substitution_df['mutation'].apply(lambda x: x[-1])

    # Initialize BINDING and PTM columns with default False values
    substitution_df['BINDING'] = False
    substitution_df['PTM'] = False

    # Check each mutation if it falls within any binding or PTM site
    for feature in row['uniprot_features']:
        feature_type = feature['type']
        feature_category = feature['type']

        feature_begin = int(feature['begin'])
        feature_end = int(feature['end'])

        if feature_type == 'BINDING':
            # Mark positions within binding site ranges
            substitution_df['BINDING'] = substitution_df['BINDING'] | substitution_df['POS'].between(feature_begin, feature_end)
        elif feature_category == 'PTM':  # Assuming 'MOD_RES' and 'CARBOHYD' are PTM types
            # Mark positions within PTM site ranges
            substitution_df['PTM'] = substitution_df['PTM'] | substitution_df['POS'].between(feature_begin, feature_end)

    return substitution_df

substitution_df = pd.concat(DMS_summary.apply(process_DMS, axis=1).to_list())


In [10]:
substitution_df

,UniProt_Accession_id,mutation,POS,REF,ALT,BINDING,PTM
0,P05067,A673C,673,A,C,False,False
1,P05067,A673D,673,A,D,False,False
2,P05067,A673E,673,A,E,False,False
127,P05067,A673F,673,A,F,False,False
128,P05067,A673G,673,A,G,False,False
...,...,...,...,...,...,...,...
10070,P46937,R203H,203,R,H,False,False
10071,P46937,R203G,203,R,G,False,False
10072,P46937,R203D,203,R,D,False,False
10073,P46937,R203S,203,R,S,False,False
